# ChIP AML PiPeline V2

In [4]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
sys.path.insert(0, '..')
import epigenetics.ChIP_helper as chiphelper
import Helper as helper
import igv
import numpy as np
import pyBigWig
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## adding the data bucket to path

In [ ]:
! gcsfuse --only-dir Chip_AML jkobject data/seqs

## doing nextflow analysis

In [ ]:
singleend, pairedend = chiphelper.extractPairedSingleEndFrom('data/seqs')

## Pipeline

![](images/gcpjup.png)


- Raw read QC (FastQC)
- Adapter trimming (Trim Galore!)
- Alignment (BWA)
- Mark duplicates (picard)
- Merge alignments from multiple libraries of the same sample (picard)
- Re-mark duplicates (picard)
- Filtering to remove: blacklisted regions, duplicates, primary alignments,unmapped,multiple locations, containing >  4 mismatches, insert size > 2kb, map to different chromosomes 
- Alignment-level QC and estimation of library complexity (picard, Preseq)
- Create normalised bigWig files scaled to 1 million mapped reads (BEDTools, bedGraphToBigWig)
- Generate gene-body meta-profile from bigWig files (deepTools)
- Calculate genome-wide IP enrichment relative to control (deepTools)
- Calculate strand cross-correlation peak and ChIP-seq quality measures including NSC and RSC (phantompeakqualtools)
- Call broad/narrow peaks (MACS2)
- Annotate peaks relative to gene features (HOMER)
- Create consensus peakset across all samples and create tabular file to aid in the filtering of the data (BEDTools)
- Count reads in consensus peaks (featureCounts)

![](images/nfcore.png)


In [ ]:
! nextflow cloud create 'JKcluster' -c 4

In [ ]:
! nextflow cloud create jkcluster -c "nextflow/nextflow.config" 40 && \
nextflow nf-core/chipseq -c "nextflow/nextflow.config" \
--singleEnd \
--seq_center 'DFCI' \
--email 'jkobject@gmail.com' \
--bucket-dir 'gs://jkobject/Chip_AML/nextflow/CHIPprocess_2/' \
--keyfile '~/jkobject-b6f1adaffcb8.json' \
--projectname 'jkobject' \
--zone 'us-east1-b' \
--skipDiffAnalysis \
--narrowPeak \
--design "nextflow/design.csv" \ 
--genome 'GRCh38' \
--profile gcp \
--resume \
--skipPreseq \
--max_cpus 8 && \
nextflow cloud shutdown jkclustert

## displayingPeaks

In [32]:
!gsutil -m cp -r gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/ ../data

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_1_7_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_1_7_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_1_7_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_1_7_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_3_351_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_3_351_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_3_351_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_3_351_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_4_352_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/D0_4_352_R1_p

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/DFAM71927V3_1_750_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/DFAM71927V3_1_751_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/DFAM71927V3_1_751_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/DFAM71927V3_1_751_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/DFAM71927V3_1_751_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/HEL9217_1_593_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/HEL9217_1_593_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/HEL9217_1_593_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/HEL9217_1_593_R1_summits.bed...
Copying gs://amlproject

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MOLM13_1_48_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_1_591_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_2_592_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_1_591_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_3_777_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_3_777_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_1_591_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_3_777_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MONOMAC1_4_778_R1_peaks.narrowPeak...
Copying gs://amlproject/C

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_138_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_293_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_138_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_293_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_293_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_293_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_319_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_320_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_319_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/m

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_71_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_71_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_71_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_71_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_72_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_72_R1_peaks.narrowPeak...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_72_R1_peaks.xls...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_72_R1_summits.bed...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/MV411_1_767_R1_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/ma

...

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D0_3_351_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_4_353_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_1_154_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_1_156_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D0_4_352_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_1_2_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_1_155_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_2_27_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/D9_1_2_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chi

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MOLM13_1_48_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MOLM13_1_48_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC1_2_592_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC1_2_592_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC1_3_777_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC1_4_778_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC6_1_196_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC1_3_777_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MONOMAC1_4_778_R

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_70_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_71_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_71_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_72_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_72_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_767_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_767_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_771_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_1_773_R1_peaks.FRiP_mqc.tsv...
Co

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_5_589_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_6_763_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_5_589_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_6_763_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_7_714_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_7_764_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_7_714_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_7_764_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/MV411_8_715_R1_peaks.FRiP_mqc.tsv.

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/TF1_2_204_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/TF1_2_204_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/U937_1_73_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/U937_2_90_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/U937_3_213_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/U937_3_213_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/SKNO1_2_637_R1_peaks.count_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/UCSDAML1_2_188_R1_peaks.FRiP_mqc.tsv...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/macs/narrowPeak/qc/UCSDAML1_2_188_R1_peaks.count_mqc.tsv...
C

In [28]:
!gsutil -m cp -r gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/ ../data

Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/IGG_MV411_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/INPUT_MV411_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-DMSO-8h_1_728_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-DMSO-8h_2_724_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-DMSO-8h_2_734_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-DMSO_1_183_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-IRF2BP2-CT-GFP-8_1_762_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-MEF2D-NT-SC-63_1_760_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411-PU1-NT-SC-1_1_761_R1.mLb.clN.bigWig...
Copying gs://amlproject/Chip/results/bwa/mergedLibrary/bigwig/MV411_1_106_

Resuming download for ../data/bigwig/INPUT_MV411_R1.mLb.clN.bigWig component 0  
Resuming download for ../data/bigwig/INPUT_MV411_R1.mLb.clN.bigWig component 1
| [86/86 files][ 20.2 GiB/ 20.2 GiB] 100% Done 475.7 MiB/s ETA 00:00:00         
Operation completed over 86 objects/20.2 GiB.                                    


In [40]:
!cp ../data/narrowPeak/*MV411*.narrowPeak ../data/MV4narrow

In [100]:
peaks = !ls ../data/MV4narrow
peaks

['MV411_1_106_R1_peaks.narrowPeak',
 'MV411_1_115_R1_peaks.narrowPeak',
 'MV411_1_117_R1_peaks.narrowPeak',
 'MV411_1_118_R1_peaks.narrowPeak',
 'MV411_1_137_R1_peaks.narrowPeak',
 'MV411_1_138_R1_peaks.narrowPeak',
 'MV411_1_293_R1_peaks.narrowPeak',
 'MV411_1_319_R1_peaks.narrowPeak',
 'MV411_1_320_R1_peaks.narrowPeak',
 'MV411_1_324_R1_peaks.narrowPeak',
 'MV411_1_357_R1_peaks.narrowPeak',
 'MV411_1_359_R1_peaks.narrowPeak',
 'MV411_1_430_R1_peaks.narrowPeak',
 'MV411_1_433_R1_peaks.narrowPeak',
 'MV411_1_565_R1_peaks.narrowPeak',
 'MV411_1_570_R1_peaks.narrowPeak',
 'MV411_1_577_R1_peaks.narrowPeak',
 'MV411_1_581_R1_peaks.narrowPeak',
 'MV411_1_582_R1_peaks.narrowPeak',
 'MV411_1_583_R1_peaks.narrowPeak',
 'MV411_1_587_R1_peaks.narrowPeak',
 'MV411_1_590_R1_peaks.narrowPeak',
 'MV411_1_601_R1_peaks.narrowPeak',
 'MV411_1_639_R1_peaks.narrowPeak',
 'MV411_1_640_R1_peaks.narrowPeak',
 'MV411_1_650_R1_peaks.narrowPeak',
 'MV411_1_70_R1_peaks.narrowPeak',
 'MV411_1_71_R1_peaks.narrowP

In [101]:
bindings = chiphelper.loadNarrowPeaks('../data/MV4narrow/', isMacs=False,skiprows=1)

In [102]:
bindings.to_csv('temp/bindings.bed',sep='\t',index=False,header=False)

In [ ]:
bindings= pd.read_csv('temp/bindings.bed',sep='\t',header=None, 
                      names=["chrom","start","end","peak_number","size","foldchange","-log10pvalue",
                             "-log10qvalue","relative_summit_pos","name"])

In [50]:
SEgenes = pd.read_csv('data/SEgenes.csv')
CTF = pd.read_csv('data/CTF.csv', header=None)[0].tolist()

In [53]:
CTF.extend(['GATA2','IKZF1','LYL1' ,'PU1','SMC1'])
CTF

['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'GSE1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'IKAROS',
 'GFI1',
 'LMO2',
 'CEBPA',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'ETV6',
 'LDB1',
 'RUNX2',
 'SP1',
 'ZMYND8',
 'GATA2',
 'IKZF1',
 'LYL1',
 'PU1',
 'SMC1']

In [99]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()
gsheet

,id,name,replicate,bams,quality,protein,matching input name,processed,bigwig,broadPeaks,...,individual QC,FLAGTOCOMP,total reads,mapped reads (hg38),unique mapped reads (hg38),multiple mapped reads (hg38),mapped read (droso),unique mapped reads(droso),scaling factor,previous name
0,mp100,INPUT_U937,1,gs://amlproject/Chip_AML/Chip/mp100.fastq.gz,NaN,INPUT,NaN,Y,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mp101,INPUT_NOMO1,1,gs://amlproject/Chip_AML/Chip/mp101.fastq.gz,NaN,INPUT,NaN,Y,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mp102,INPUT_UT7,1,gs://amlproject/Chip_AML/Chip/mp102.fastq.gz,NaN,INPUT,NaN,Y,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mp106,MV411,1,gs://amlproject/Chip_AML/Chip/mp106.fastq.gz,x,MYB,INPUT_MV411,Y,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp109,M6,1,gs://amlproject/Chip_AML/Chip/mp109.fastq.gz,x,CEBPA,INPUT_M6,Y,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,mp826,MV411-IRF2BP2++-VHL-6h,2,NaN,NaN,H3K4me1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174092.0,0.936120,NaN
255,mp827,MV411-IRF2BP2++-VHL-6h,1,NaN,NaN,H3K4me3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,187078.0,0.618929,NaN
256,mp828,MV411-IRF2BP2++-VHL-6h,2,NaN,NaN,H3K4me3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,495924.0,0.270745,NaN
257,mp829,MV411-IRF2BP2++-VHL-6h,1,NaN,NaN,H3K79me2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225315.0,0.762888,NaN


In [103]:
rename= {}
for peak in peaks:
    val = gsheet[gsheet.id=='mp'+peak.split('_R1')[0][-3:].split('_')[-1]]
    rename[peak]= val.protein.values[0]+"_"+str(val.replicate.values[0])

In [104]:
rename

{'MV411_1_106_R1_peaks.narrowPeak': 'MYB_1',
 'MV411_1_115_R1_peaks.narrowPeak': 'PU1_1',
 'MV411_1_117_R1_peaks.narrowPeak': 'POLII_1',
 'MV411_1_118_R1_peaks.narrowPeak': 'SP1_1',
 'MV411_1_137_R1_peaks.narrowPeak': 'H3K4me3_1',
 'MV411_1_138_R1_peaks.narrowPeak': 'H3K36me3_1',
 'MV411_1_293_R1_peaks.narrowPeak': 'H3K79me2_1',
 'MV411_1_319_R1_peaks.narrowPeak': 'FLI1_1',
 'MV411_1_320_R1_peaks.narrowPeak': 'ZEB2_1',
 'MV411_1_324_R1_peaks.narrowPeak': 'MEF2D_1',
 'MV411_1_357_R1_peaks.narrowPeak': 'ZMYND8_1',
 'MV411_1_359_R1_peaks.narrowPeak': 'LMO2_1',
 'MV411_1_430_R1_peaks.narrowPeak': 'IRF2BP2_1',
 'MV411_1_433_R1_peaks.narrowPeak': 'GSE1_1',
 'MV411_1_565_R1_peaks.narrowPeak': 'CDK13_1',
 'MV411_1_570_R1_peaks.narrowPeak': 'CTCF_1',
 'MV411_1_577_R1_peaks.narrowPeak': 'SMC1_1',
 'MV411_1_581_R1_peaks.narrowPeak': 'MEF2C_1',
 'MV411_1_582_R1_peaks.narrowPeak': 'MEIS1_1',
 'MV411_1_583_R1_peaks.narrowPeak': 'ELF2_1',
 'MV411_1_587_R1_peaks.narrowPeak': 'IKZF1_1',
 'MV411_1_590_R

In [107]:
bindings['name'] = [rename[i+'.narrowPeak'] for i in bindings['name']]

In [ ]:
%matplotlib inline
peakstoplot, figure = chiphelper.computePeaksAt(
    bindings[bindings.name=='MYC_1'],
    bigwigs=[
    "MYC_R1.mLb.clN.bigWig",
    "MYC_R2.mLb.clN.bigWig",
    "CEBPA_R1.mLb.clN.bigWig",
    "CEBPA_R2.mLb.clN.bigWig",
], folder='data/bigwig/', window=600,numpeaks=2000, width=10,length=10)

In [ ]:
%matplotlib inline
peakstoplot, figure = chiphelper.computePeaksAt(
    bindings[bindings.name=='CEBPA_R1'],
    bigwigs=[
    "MYC_R1.mLb.clN.bigWig",
    "MYC_R2.mLb.clN.bigWig",
    "CEBPA_R1.mLb.clN.bigWig",
    "CEBPA_R2.mLb.clN.bigWig",
], folder='data/bigwig/', window=600,numpeaks=8000, width=10,length=10)

In [ ]:
%matplotlib inline
peakstoplot, figure = chiphelper.computePeaksAt(
    bindings[bindings.name=='CEBPA_R2'],
    bigwigs=[
    "MYC_R1.mLb.clN.bigWig",
    "MYC_R2.mLb.clN.bigWig",
    "CEBPA_R1.mLb.clN.bigWig",
    "CEBPA_R2.mLb.clN.bigWig",
], folder='data/bigwig/', window=600,numpeaks=10000, width=10,length=10)

## Marking each TF with a different quality score

## merging duplicates

In [ ]:
replicates = chiphelper.findReplicates(folder='data/seqs/results/bwa/', sep='_', namings='_R([0-9])',namepos=0)

# we do a visual inspection of the features and and look at QCs

### [igv tracks](https://igv.org/app/?sessionURL=blob:3Z3rU9rcFsb_lU6.vO.Zg0EIcvGbolinYB0vp5czHSeEGFIhscmOaJ3.790hwMbzsrZZ9rTJIx.YAfIk62HxY7Mfcnk0IvfajdzAcY3dR8MfGbvG2Kt1jIoR2FP5nPE2mdrBm7.7Z92x1aqmr_1Lvnhtx8K.POuniwtxG.9Wq7FljhJ7Ip93bswk3nLlIls1057a38PAnsWmE06rvndnDqPQHvlBLHyRCNcMI6_quUE4deNq7H6bb2J.Z843IjfmByP3_o9sTN77coPOgwiHdjD6jdtMN7EvN2GKe2H8qBiT0EliY_e_hj2ZGF8qhojkVtInHg3xcJs2QkqTeZ8qRhiN3MjYlRtvdVrNjlWzGu1m3arttNx_W9tt2bvw_NYPgnQhESXuj8qjkUQTuRIvNfL1Jhx.dR1R7Y7926u9Qb8auXEyEXF1OLOrUzfy3FHfH0Z29FAd.t7M96q9_nHt6qxmTvtD05mcmPLpD773pDdBMpmsSttefX5IpRNOQrmkEXnDv7cr25Vaq51.suyJ6G5.Zez63lgYu5ZcuZ2I8NyxJ3ILqcGKMZOFhLNeEjjCDwMpnrp2IEV3fuwP_YkvHj7Ml5CvbNXk85PQW.ivZUvlCv7nLZMfcX_iPuchDpPIcS.yBqWCFI0wmtqyTCN76.QziwZmD9LKY1U5qzWBHUXh7NS1b5YNubqVD2JTvUC3pLZqyV9T28mEf_1KH.pP.7B4H__RCCdMAqHpRI43fpNL7VufLZmq1NtvB0Eo7HlRFWNq35.FM8nXzrZ0IeXCjdJ1zRF0xlE4DWMJrFxSkusaX36doKMej6D6ql2kEogg0kNxBC1KYhBkARKkcYlG0PHJ6eUFB6HGql.0FIgh2sQfhmhjdwaHvfoBpzs7q.7QUqDu0CaK.4pb1sT4jmsCfsfpbKJ9yZ1esn4ltFbtooRACFEWigMoq4iBTzr3QsOHNokGz_kpC56amhJRSiB6KAvF0ZNVxJmiqkkPDj60SzR8Ph_u11n8qAkRKQUCiPRQHEGLkjgIqUkPDkIam2gMdQ_3T_dYEKl5EK0Foog2URxGy5o4HCFOhHQ.QUGq5wdJzYVoLR5IG0wUDlKdAxLilEjnEw6ki26PNSB1FEeUFAkjykOBFGUlMSBKP_J4ENE20Rg6GJy_v2qPr95a7.ot.f5z_sdTIUOetQCRlcdOcZD9szoOb4hBRD7HqOgN9j6ysFPZxHNrAESOslI8blllHNQQA4vn3UJi9pLhTcUXz64CDbQSD2wvG9UQM40cdtFYO.z3WDl7XSUbpBSILdJDcUwtSuKwhBhraGxiMpQ_HayrVIOUwjFUqmxwURJnf0rEVENjE46hi_80OeOQpaIMUorEEOWhQIaykjgMISYVGpuYDOUfhyyVS5BSOIbKNQ5lJXEYQowgNDYxGbLyM6QyB1IKx9AGD0UzZHEYQswWNDbhDi_bu9hjhQqWChVoLdIRZqSJAg8xW9TE4QgxV9D5BAWJ8YtOJQu0Fg.kUv2mW9bEAKmBGC7ofMKBdH7IOhqjodIFUoqEEeWhQIqykjgQIaYLGpuYDOUfixoqXSClcAyVayTKSuIwhJguaGyiMfSCnRoaKmB4JfszlHNXhhftxdBATBpe1w4Mx0dHLJxU1EApkU7DQVgoDqOsIg5CiCED7RIOn3efe7ypkYoYaC0SQqSJAiFa1MTAaAcxYtD5BAUp__xoR2UMtBYPpFLNkJY1cUBCjBl0PuFAOuvV909Z_x_tqKhBp0aCSWOjQJxWVXGAQswc9E5hkWKMTip20KkRkSrXCLWqioMUYuygdwqLVP5dhXbWogeNGhGpUu0wpKriIAUZQ2idwiLVyI_UWhihUSMitcFGCZBqcE7fChlJaJ0CItXmTKSaa5kEJcWCabOHQklKS.JgBBlI0DbRGOoP3rPCiKYKI0gpEEOkh.IYWpTEYQgxg9DYxGQof_rQVOkDKYVjqFS5w6IkDkOIoYPGJhxDn_qsP2qbKm4gpUgMUR4KZCgricMQYsqgsYnGEPPEXU0VL7yCE3aV70Rd7BN0tRAjhddzYq7B4QHvijAqTiClUJdVIjwUeVWleUkchBDjBI1NPIZ69S4LIpUn0Fqwi5NtNlHsxcnSmjgcIUYKOp.gIOUPFVpPr_K3WYsHUqlihWVNHJBQL_NH.QQFKf.ODC2VLNBaPJBKtRPDsiYOSIjhgs4nHkjH57z50Vq.QGqhQKJMFAlSVhPnwpmQKYPGJxxIn_Y5GLXXYgZCiQQRYaFAhOYVcQCCzBhIl5D45J8XtdcCBkKJhk.55kTzijj4QEYLpEs8fFj5XHstViCUUPiULpubV8TBBzJQIF1C4sMYfdbCBEKJhk_JRh9mHteGjBFIl2j4nL7vHx.zxh8VItBaIIRoE8VBtKyJgVEHMUTQ.UQD6ezy5CMrjeuoGIHWAoFEmygOpGVNHJAQwwSdT0SQWEdAdFSgQGvBQCrbMRDLmjggIcYKOp9oIJ0PurwBSSULpBQII9JDcRQtSuJAhBguaGyiMfT5cJ91JFFHxQukFIgh0kNxDC1K4jCEmDBobMIxNPh0csA6LryjMgaNGIkj2kWBJC2LYrBUSz80eDDpnKLilH9Qqm2rqEGjBuTpt41My.mAex27394cydbF6.uJ3Ot5OyuLXo6FuE3bGVvmKJHlC9u5MZN4y7VjsVUz7an9PQzsWWw64bTqe3dmGHmmXIP8ZMTVsVfryLZfp1sx4zAS7sgU98L0vj9p6e_aiCmG_sa.bMlPTeROwzt7uNYW9UWQ3vRtokxt6tH_Ba4vP34C)

### [multiQC](http://35.184.213.1:8888/view/data/results/multiqc/multiqc_report.html)

### process: 

look at all t with a very low frip score as noted by encode. 

look at all peaks tracks together and see for location of intense co binding. 

- if we can discern peaks and if, for some reasons, some good peaks are not called by macs. 
- if looks good and we can see a lot of peaks. 
- if a lot of noise but seems consistent with replicates. 
- if just seems to have very few peaks.

Validate still but flag as potentially bad.

Else remove.

### results:

In [ ]:
toremove=['ETV6_R3'
'ELF2_R2'
'MEF2D_R1'
'RUNX2_R1'
'IKZF1_R2']

bad=[
"LMO2_R1",
"LMO2_R2",
"IRF2BP2_R2",
"IRF2BP2_R4",
"MEF2C_R1",
"MEF2C_R2",
"MEF2C_R3",
"MEF2D_R2",
"MEF2D_R3",
"ZMYND8_R2",
"GATA2_R1",
"GSE1_R1",
"GSE1_R2",
"ZEB2_R1",
"ZEB2_R1",
"GFI1_R1",
"POL2",
"IRF2BP2_R3"]

In [ ]:
bindings['name']= [i[:-6] for i in bindings['name']]
bindings['tf'] = [i.split('_')[0] for i in bindings['name']]
bindings=bindings[~bindings.name.isin(toremove)]

In [ ]:
bindings

In [ ]:
replicates

### Merge

**If A U B  > ½ A and #A = #B** *we merge peaks and flag for bam merge

**If 0 < A U B  < ½ A and #A >> #B**  *we keep only A and flag for bam merge

- Not so good of an overlap. 
- Most of the time, one will have much more peak than the other

all is about choosing values of:
- how much is enough overlap, 
- how much read is enough to say we found an uncalled peak...


In [ ]:
%matplotlib inline
mergedpeak, tomergebam, remove = chiphelper.mergeReplicatePeaks(bindings,
                            replicates,'data/bigwig/',markedasbad=bad,
                            window=200, mincov=4, doPlot=True, cov={}, use='max',
                            MINOVERLAP=0.5, SIZECUTOFF=0.7, mergewindow=50)

In [ ]:
ELF2
GATA2
IKZF1
ZEB2

In [ ]:
tomergebam = [['CEBPA_R2', 'CEBPA_R1'],
 ['IRF2BP2_R4', 'IRF2BP2_R1'],
 ['IRF2BP2_R4', 'IRF2BP2_R2'],
 ['IRF2BP2_R4', 'IRF2BP2_R3'],
 ['LMO2_R1', 'LMO2_R2'],
 ['MEF2C_R1', 'MEF2C_R2'],
 ['MEF2C_R1', 'MEF2C_R3'],
 ['MYB_R1', 'MYB_R2'],
 ['MYC_R1', 'MYC_R2'],
 ['ZMYND8_R1', 'ZMYND8_R2']]

In [ ]:
remove = ['DMSO','CTCF','ETV6','GSE1']

In [ ]:
mergedpeak = pd.concat([peaks for _, peaks in mergedpeak.items()]).drop(['tf','size'],1)

In [ ]:
mergedpeak.drop(['tf','size'],1).to_csv('temp/merged.csv')

In [ ]:
mergedpeak.name = [i.split('_')[0] for i in mergedpeak.name]

## show replicates overlap

### sorting and removing samples

In [ ]:
bigwigs=os.listdir('data/bigwig/')
for val in bigwigs:
    for v in remove + toremove + ['scale','POLII','IGG','CTCF','INPUT']:
        if v in val:
            bigwigs.remove(val)
            break
bigwigs = ['data/bigwig/'+ i for i in bigwigs]

In [ ]:
merged = chiphelper.simpleMergedPeaks(mergedpeak[~mergedpeak.name.isin(
    toremove+remove+['scale','POLII','IGG','CTCF','INPUT'])], window=200)

In [ ]:
merged.to_csv('temp/merged.csv')

In [ ]:
merged = pd.read_csv('temp/merged.csv').drop('Unnamed: 0',1)

In [ ]:
bigwigs

## Co Binding Matrix

Look at AUC for all ChIPs over all peaks of all ChIPs

In [ ]:
merged[merged.columns[8:]].sum(0)

In [ ]:
i = merged[merged.columns[8:]].sum(1)
i.max(),i.mean(),i.min()

In [ ]:
cor = chiphelper.createCorrMatrix(merged, bigwigs, window=10)

In [ ]:
np.save('temp/corr.npy', cor)

In [ ]:
cor = np.load('temp/corr.npy')

In [ ]:
cor.mean(0).min()

In [ ]:
cor.shape

## dropping weird samples: IKZF1_R1, GATA2_R2, IRF2BP2_R4

In [ ]:
weird = [29,20,16]
cor  = np.delete(cor, weird, 0)
bigwigs = [a for i,a in enumerate(bigwigs) if i not in weird]

In [ ]:
PCS=30
TOTVAR=50000

In [ ]:
cor = np.log2(1+cor)
a = cor.var(0).argsort()
subcor = cor[:,a[-TOTVAR:]]

In [ ]:
subcor.shape

In [ ]:
subcor = PCA(PCS).fit_transform(subcor)

In [ ]:
%matplotlib inline
cmaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
         'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
         'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
import seaborn as sns
from matplotlib import pyplot 
fig, ax = pyplot.subplots(figsize=(20,15))
sns.heatmap(ax=ax, data=subcor,xticklabels=False, cmap=cmaps[-4],
            yticklabels=[i.split('.')[-4].split('/')[-1] for i in bigwigs],
            cbar_kws={"orientation": "horizontal"},
            vmax=0.7).set_title('PCs of each binding profile over conscensus peakset')
#fig.savefig("temp/co_occupancy_matrix.png")

In [ ]:
fig.savefig('PCs_binding_peakset.png')

In [ ]:
model = AgglomerativeClustering(n_clusters=5,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(subcor)
ii = itertools.count(cor.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]

In [ ]:
labels

## clustering

I have tried gaussian mixtures and Agglomerative clustering algorithm. Only the second can create a hierarchical clustering.

It seems that gaussian mixture makes more sense given the data we have, for now, is more "homogeneous". 

**I am still not so happy with the clustering.** It can be because of the how much importance, outlier values and the high number of noisy values from locations with no peaks.

We can use similar methods to RNAseq to improve this (clamping values, log transform, first round of PCA..)


In [ ]:
labels = GaussianMixture(n_components=2, covariance_type='diag').fit_predict(subcor)

In [ ]:
names = np.array([i.split('.')[-4].split('/')[-1] for i in bigwigs])
sort = labels.argsort()
p = helper.plotCorrelationMatrix(data=cor[sort],
                            names=names[sort],
                            colors=labels[sort],
                            title="correlation between TF occupancy",
                            interactive=True)

In [ ]:
show(p)

In [ ]:
p = helper.scatter(TSNE(2,5).fit_transform(subcor),labels=names, colors=labels)

In [ ]:
show(p)

# Next steps

## annotatePeaks

### based on closest expressed gene

### based on the ABC model

![](images/ABCtitle.png)

They tested a new model based on and validated by CRISPRi-FlowFISH which is basically able to find enhancer mapping to genes. 
They used it to compute their model's Accuracy and found a 70% accuracy compared to less than 50% for closest expressed gene. 

Way to integrate our HiC data (need ATAC-seq like data as well, but openly available) 


![](images/ABCmodel.png)

In [ ]:
Helper.scatter(TSNE(2,5).fit_transform(data.T), labels=zones.columns[11:],colors=labels)

## Compute the CRC and merge with gene assignement

~1500 lines of code. Seems to be a slightly updated version from the code I used the first time which was from their originial CRC paper. 

There is not a lot of documentation but it was just updated last week and might continue like that

![](images/CRCpres.png)

## find set of TFs that explain the most cobindings 

## relate to gene depencies

## what else?

### Compare data with other labs (H3K27, HiC..)

we need to redo everything for similar normal cell type, getting TFs based on the CRC (find it with CRCmapper or on litterature)